In [1]:
from pdf2image import convert_from_path
import cv2
import numpy as np


Note: you may need to restart the kernel to use updated packages.


In [6]:
# def count_hollow_circles(pdf_path):
#     # Convert PDF to images
#     images = convert_from_path(pdf_path)

#     # Initialize a counter for hollow circles
#     total_hollow_circles = 0

#     for page_number, image in enumerate(images):
#         # Convert the image to grayscale
#         gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

#         # Apply GaussianBlur to reduce noise and improve circle detection
#         blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#         # Use Hough Circle Transform to detect circles
#         circles = cv2.HoughCircles(
#             blurred,
#             cv2.HOUGH_GRADIENT,
#             dp=1,
#             minDist=20,
#             param1=50,
#             param2=30,
#             minRadius=10,
#             maxRadius=50
#         )

#         if circles is not None:
#             # Convert the circle coordinates to integers
#             circles = np.uint16(np.around(circles))

#             for i in circles[0, :]:
#                 # Draw the circle on the image
#                 cv2.circle(image, (i[0], i[1]), i[2], (0, 255, 0), 2)

#                 # Check if the circle is hollow (implement your logic here)
#                 # For example, you can check if the average pixel intensity inside the circle is below a certain threshold
#                 is_hollow = True  # Replace this with your actual logic

#                 if is_hollow:
#                     total_hollow_circles += 1

#     return total_hollow_circles

# # Example usage
# pdf_path = 'one.pdf'
# hollow_circle_count = count_hollow_circles(pdf_path)
# print(f'Total number of hollow circles: {hollow_circle_count}')


In [2]:
# from pdf2image import convert_from_path
# import cv2
# import numpy as np

# def count_hollow_circles(pdf_path,poppler_path):
#     # Convert PDF to images
#     images = convert_from_path(pdf_path, poppler_path=poppler_path)

#     # Initialize a counter for hollow circles
#     total_hollow_circles = 0

#     for page_number, image in enumerate(images):
#         # Convert the image to grayscale
#         gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

#         # Apply GaussianBlur to reduce noise and improve circle detection
#         blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#         # Use Hough Circle Transform to detect circles
#         circles = cv2.HoughCircles(
#             blurred,
#             cv2.HOUGH_GRADIENT,
#             dp=1,
#             minDist=20,
#             param1=50,
#             param2=30,
#             minRadius=10,
#             maxRadius=50
#         )

#         if circles is not None:
#             # Convert the circle coordinates to integers
#             circles = np.uint16(np.around(circles))

#             for i in circles[0, :]:
#                 # Draw the circle on the image
#                 cv2.circle(image, (i[0], i[1]), i[2], (0, 255, 0), 2)

#                 # Check if the circle is hollow (implement your logic here)
#                 # For example, you can check if the average pixel intensity inside the circle is below a certain threshold
#                 is_hollow = True  # Replace this with your actual logic

#                 if is_hollow:
#                     total_hollow_circles += 1

#     return total_hollow_circles

# # Example usage
# pdf_path = 'one.pdf'
# poppler_path = r'C:\Users\116317\Downloads\poppler-23.12.0\poppler-23.12.0'
# hollow_circle_count = count_hollow_circles(pdf_path, poppler_path)
# print(f'Total number of hollow circles: {hollow_circle_count}')


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [14]:
import os

# Specify the path to the Poppler binary directory
poppler_path = r'C:\Users\116317\Downloads\poppler-23.12.0\poppler-23.12.0'

# Add Poppler to the system PATH
os.environ['PATH'] += ';' + poppler_path

In [2]:
from pdf2image import convert_from_path
import cv2
import numpy as np

def count_hollow_circles(pdf_path):
    # Convert PDF to images
    images = convert_from_path(pdf_path)

    # Initialize a counter for hollow circles
    total_hollow_circles = 0

    for page_number, image in enumerate(images):
        # Convert the image to a NumPy array
        np_image = np.array(image)

        # Convert the image to grayscale
        gray = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)

        # Apply GaussianBlur to reduce noise and improve circle detection
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Use Hough Circle Transform to detect circles
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=20,
            param1=50,
            param2=30,
            minRadius=10,
            maxRadius=50
        )

        if circles is not None:
            # Convert the circle coordinates to integers
            circles = np.uint16(np.around(circles))

            for i in circles[0, :]:
                # Draw the circle on the image
                cv2.circle(np_image, (i[0], i[1]), i[2], (0, 255, 0), 2)

                # Check if the circle is hollow (implement your logic here)
                # For example, you can check if the average pixel intensity inside the circle is below a certain threshold
                is_hollow = True  # Replace this with your actual logic

                if is_hollow:
                    total_hollow_circles += 1

    return total_hollow_circles

# Example usage
pdf_path = 'one.pdf'
hollow_circle_count = count_hollow_circles(pdf_path)
print(f'Total number of hollow circles: {hollow_circle_count}')


Total number of hollow circles: 43


In [6]:
from pdf2image import convert_from_path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


def count_hollow_circles(pdf_path, output_folder='circle_snippets'):
    # Convert PDF to images
    images = convert_from_path(pdf_path)

    # Initialize a counter for hollow circles
    total_hollow_circles = 0

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    for page_number, image in enumerate(images):
        # Convert the image to a NumPy array
        np_image = np.array(image)

        # Convert the image to grayscale
        gray = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)
        np_image1 = gray
        # Apply GaussianBlur to reduce noise and improve circle detection
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Use Hough Circle Transform to detect circles
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=20,
            param1=50,
            param2=30,
            minRadius=10,
            maxRadius=50
        )

        if circles is not None:
            # Convert the circle coordinates to integers
            circles = np.uint16(np.around(circles))

            for i, circle in enumerate(circles[0, :]):
                # Draw the circle on the image
                cv2.circle(np_image, (circle[0], circle[1]), circle[2], (0, 255, 0), 2)

                # Check if the circle is hollow (implement your logic here)
                # For example, you can check if the average pixel intensity inside the circle is below a certain threshold
                is_hollow = True  # Replace this with your actual logic

                if is_hollow:
                    total_hollow_circles += 1

                    # Extract and save the circle snippet
                    x, y, r = circle[0], circle[1], circle[2]
                    circle_snippet = np_image[y - r:y + r, x - r:x + r]

                    # Save the snippet as an image
                    snippet_filename = f'{output_folder}/circle_snippet_{page_number}_{i}.png'
                    cv2.imwrite(snippet_filename, circle_snippet)

    return total_hollow_circles

# Example usage
pdf_path = 'one.pdf'
hollow_circle_count = count_hollow_circles(pdf_path)
print(f'Total number of hollow circles: {hollow_circle_count}')


Total number of hollow circles: 43
